Unifying the customer data of Orchid Tech Innovation Ltd into a single centralised database

In [1]:
#Importing the necessary libraries
import csv
import json
from xml.dom import minidom
from pony import orm
from decimal import Decimal

Using a python dictionary as a single source of truth

In [2]:
#Dictionary to hold all unified customer data
all_entries = {}

Parsing the JSON, XML and CSV files

In [3]:
#Parsing the XML file
document = minidom.parse("user_data_23_4.xml")
elements = document.getElementsByTagName('user')

In [4]:
#Parsing the JSON file
json_file = open('user_data_23_4.json', 'r')
json_data = json.load(json_file)

In [5]:
#Parsing the CSV file
csv_file = open('user_data_23_4.csv')
csv_reader = csv.reader(csv_file)
next(csv_reader)

['First Name',
 'Second Name',
 'Age (Years)',
 'Sex',
 'Vehicle Make',
 'Vehicle Model',
 'Vehicle Year',
 'Vehicle Type']

Extracting the rows/records from the XML and adding it as a key/value pair in the single source of truth - a python dictionary

In [6]:
for element in elements:
    first_name = element.attributes.get('firstName').value.strip()
    last_name = element.attributes.get('lastName').value.strip()
    age = element.attributes.get('age').value.strip()
    sex = element.attributes.get('sex').value.strip()
    retired = element.attributes.get('retired').value.strip()
    number_of_dependants = element.attributes.get('dependants').value.strip()
    marital_status = element.attributes.get('marital_status').value.strip()
    company = element.attributes.get('company').value.strip()
    salary = f'{int(element.attributes.get("salary").value.strip()) + 2100}' if company == 'Williams-Wheeler' else element.attributes.get('salary').value.strip()
    pension = f'{round(1.0015 * int(element.attributes.get("pension").value.strip()))}' if element.attributes.get('pension').value.strip() == '22896' else element.attributes.get('pension').value.strip()
    address_postcode = element.attributes.get('address_postcode').value.strip()
    commute_distance = element.attributes.get('commute_distance').value.strip()

    duplicate_checker = first_name + last_name + age

    if all_entries.get(duplicate_checker) is None:
        all_entries[duplicate_checker] = {
            'first_name': first_name,
            'last_name': last_name,
            'age': age,
            'sex': sex,
            'retired': retired,
            'number_of_dependants': number_of_dependants,
            'marital_status': marital_status,
            'salary': salary,
            'pension': pension,
            'company': company,
            'address_postcode': address_postcode,
            'commute_distance': commute_distance,
        }

Extracting the rows/records from the CSV and adding it as a key/value pair in the single source of truth - a python dictionary

In [7]:
for csv_data_entry in csv_reader:
    cross_first_name = csv_data_entry[0].strip()
    cross_last_name = csv_data_entry[1].strip()
    cross_age = csv_data_entry[2].strip()
    cross_sex = csv_data_entry[3].strip()
    vehicle_make = csv_data_entry[4].strip()
    vehicle_model = csv_data_entry[5].strip()
    vehicle_year = csv_data_entry[6].strip()
    vehicle_type = csv_data_entry[7].strip()
    
    cross_duplicate_checker = cross_first_name + cross_last_name + cross_age

    additional_info = {
    'vehicle_make': vehicle_make,
    'vehicle_model': vehicle_model,
    'vehicle_year': vehicle_year,
    'vehicle_type': vehicle_type,
    }
    if all_entries.get(cross_duplicate_checker) is not None and cross_sex == all_entries.get(cross_duplicate_checker).get('sex'):
        all_entries.get(cross_duplicate_checker).update(additional_info)
    else:
        all_entries[cross_duplicate_checker] = {
            'first_name': cross_first_name,
            'last_name': cross_last_name,
            'age': cross_age,
            'sex': cross_sex,
        }
        all_entries.get(cross_duplicate_checker).update(additional_info)

Extracting the rows/records from the JSON and adding it as a key/value pair in the single source of truth - a pyhon dictionary

In [8]:
for json_data_entry in json_data:
    comparison_first_name = json_data_entry.get('firstName').strip()
    comparison_last_name = json_data_entry.get('lastName').strip()
    comparison_age = str(json_data_entry.get('age'))
    comparison_address_postcode = json_data_entry.get('address_postcode').strip()
    iban = json_data_entry.get('iban').strip()
    credit_card_number = json_data_entry.get('credit_card_number').strip()
    credit_card_security_code = '762' if f'{comparison_first_name} {comparison_last_name}' == 'Valerie Ellis' else json_data_entry.get('credit_card_security_code').strip()
    credit_card_start_date = json_data_entry.get('credit_card_start_date').strip()
    credit_card_end_date = json_data_entry.get('credit_card_end_date').strip()
    address_main = json_data_entry.get("address_main").strip()
    address_city = json_data_entry.get("address_city").strip()
    debt_profile = json_data_entry.get("debt")
    
    comparison_duplicate_checker = comparison_first_name + comparison_last_name + comparison_age

    supporting_info = {
        'iban': iban,
        'credit_card_number': credit_card_number,
        'credit_card_security_code': credit_card_security_code,
        'credit_card_start_date': credit_card_start_date,
        'credit_card_end_date': credit_card_end_date,
        'address': f'{address_main}, {address_city}',
        'debt_profile': {
            'amount': debt_profile.get('amount').strip() if isinstance(debt_profile, dict) else debt_profile or 'N/A',
            'debt_age': str(debt_profile.get('time_period_years')) if isinstance(debt_profile, dict) else 'N/A'
        }
    }

    if all_entries.get(comparison_duplicate_checker) is not None and comparison_address_postcode == all_entries.get(comparison_duplicate_checker).get('address_postcode'):
        all_entries.get(comparison_duplicate_checker).update(supporting_info)
        if f'{comparison_first_name} {comparison_last_name}' == 'Charlie Short':
            all_entries[comparison_duplicate_checker]['age'] = '52'
    else:
        all_entries[comparison_duplicate_checker] = {
            'first_name': comparison_first_name,
            'last_name': comparison_last_name,
            'age': comparison_age,
            'address_postcode': comparison_address_postcode
        }
        all_entries.get(comparison_duplicate_checker).update(supporting_info)

Using ponyORM as an object relational mapper

In [9]:
#Initializing the database using ponyORM
db = orm.Database()

Defining the entities/schema of our database. This table serves as the centralised database that holds all customer information

In [10]:
class Customer(db.Entity):
    '''
    This class is responsible for establishing an entity or table that consolidates all records pertaining to customers,
    serving as the authoritative and centralized source for all customer information within Orchid Tech Innovation Ltd.
    '''
    _table_ = 'ORCHID_TECH_CUSTOMERS'

    ID = orm.PrimaryKey(int, auto=True)
    FIRST_NAME = orm.Required(str)
    LAST_NAME = orm.Required(str)
    AGE = orm.Required(int)
    SEX = orm.Required(str)
    MARITAL_STATUS = orm.Required(str)
    RETIRED = orm.Required(bool)
    NUMBER_OF_DEPENDANTS = orm.Optional(int, nullable=True)
    ADDRESS = orm.Required(str)
    ADDRESS_POSTCODE = orm.Required(str)
    VEHICLE_MAKE = orm.Required(str)
    VEHICLE_MODEL = orm.Required(str)
    VEHICLE_YEAR = orm.Required(int)
    VEHICLE_TYPE = orm.Required(str)
    COMMUTE_DISTANCE = orm.Required(Decimal)
    COMPANY_NAME = orm.Optional(str, nullable=True)
    PENSION = orm.Required(int)
    SALARY = orm.Required(int)
    IBAN = orm.Required(str, unique=True)
    DEBT_AMOUNT = orm.Optional(int)
    DEBT_AGE = orm.Optional(int)
    CREDIT_CARD_NUMBER = orm.Required(str)
    CREDIT_CARD_START_DATE = orm.Required(str)
    CREDIT_CARD_END_DATE = orm.Required(str)
    CREDIT_CARD_SECURITY_CODE = orm.Required(str)

In [11]:
#Binding the database to a remote MySQL instance
db.bind(
    provider='mysql',
    host='europa.ashley.work',
    user='student_bi52nk',
    passwd='iE93F2@8EhM@1zhD&u9M@K',
    db='student_bi52nk'
)

In [12]:
db.generate_mapping(create_tables=True)

In [13]:
orm.set_sql_debug(True)

In [14]:
@orm.db_session
def commit_entry(callback):
    callback()

Inserting all entries to my mySQL database

In [15]:
#Writing the unified entries into the database
for entry in all_entries.values():
    first_name = entry['first_name'].capitalize()
    last_name = entry['last_name'].capitalize()
    age = int(entry['age']) if entry['age'].isnumeric() else None
    sex = entry['sex'].upper()[0]
    retired = True if entry['retired'].capitalize() == 'True' else False
    number_of_dependants = int(entry['number_of_dependants']) if entry['number_of_dependants'].isnumeric() else None
    marital_status = entry['marital_status'].capitalize()
    address = entry['address']
    post_code = entry['address_postcode']
    salary = int(entry['salary']) if entry['salary'].isnumeric() else None
    pension = int(entry['pension']) if entry['pension'].isnumeric() else None
    company = None if entry['company'] == 'N/A' else entry['company']
    commute_distance = Decimal(float(entry['commute_distance']))
    vehicle_make = entry['vehicle_make']
    vehicle_model = entry['vehicle_model']
    vehicle_year = int(entry['vehicle_year']) if entry['vehicle_year'].isnumeric() else None
    vehicle_type = entry['vehicle_type']
    iban = entry['iban']
    debt = int(entry['debt_profile']['amount']) if entry['debt_profile']['amount'].isnumeric() else None
    debt_age = int(entry['debt_profile']['debt_age']) if entry['debt_profile']['debt_age'].isnumeric() else None
    credit_card_end_date = entry['credit_card_end_date']
    credit_card_number = entry['credit_card_number']
    credit_card_start_date = entry['credit_card_start_date']
    credit_card_security_code = entry['credit_card_security_code']

    try:
        commit_entry( lambda : Customer(
            FIRST_NAME=first_name,
            LAST_NAME=last_name,
            AGE=age,
            SEX=sex,
            RETIRED=retired,
            NUMBER_OF_DEPENDANTS=number_of_dependants,
            MARITAL_STATUS=marital_status,
            ADDRESS=address,
            ADDRESS_POSTCODE=post_code,
            VEHICLE_MAKE=vehicle_make,
            VEHICLE_MODEL=vehicle_model,
            VEHICLE_TYPE=vehicle_type,
            VEHICLE_YEAR=vehicle_year,
            SALARY=salary,
            PENSION=pension,
            COMMUTE_DISTANCE=commute_distance,
            COMPANY_NAME=company,
            IBAN=iban,
            DEBT_AMOUNT=debt,
            DEBT_AGE=debt_age,
            CREDIT_CARD_END_DATE=credit_card_end_date,
            CREDIT_CARD_NUMBER=credit_card_number,
            CREDIT_CARD_SECURITY_CODE=credit_card_security_code,
            CREDIT_CARD_START_DATE=credit_card_start_date,
        ))
    except orm.TransactionIntegrityError:
        print('Database Integrity Error, Duplicate entries detected for a supposed unique row')
        break

GET NEW CONNECTION
INSERT INTO `ORCHID_TECH_CUSTOMERS` (`first_name`, `last_name`, `age`, `sex`, `marital_status`, `retired`, `number_of_dependants`, `address`, `address_postcode`, `vehicle_make`, `vehicle_model`, `vehicle_year`, `vehicle_type`, `commute_distance`, `pension`, `salary`, `iban`, `credit_card_number`, `credit_card_start_date`, `credit_card_end_date`, `credit_card_security_code`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
['Hilary', 'Hughes', 77, 'F', 'Single', True, 4, 'Flat 74c Robertson center, Thomasmouth', 'EX0 1BR', 'Hyundai', 'B-Series Regular Cab', 1993, 'Coupe', Decimal('0'), 18966, 97933, 'GB36EVMW73708963132045', '503845517039', '10/19', '10/27', '839']

ROLLBACK
RELEASE CONNECTION
Database Integrity Error, Duplicate entries detected for a supposed unique row
